# Decision Trees
This notebook is higly inspired by **[AssemblyAI](https://www.youtube.com/@AssemblyAI)=>[How to Implement Decision Trees from scratch with Python](https://youtu.be/NxEHSAfFlK8) KUDOS TO THE TEACHERS**

So what are `Decision Trees...?`

<img src  = "https://www.mihaileric.com/static/decision-tree-meme-29e8e977fa4a9dc3fa25e37d3507f09e-bc7bc.jpeg">

Suppose you have a dataset like this 

|Age|Eligibe To Vote
|---|---
|13|No
|15|No
|18|Yes
|0|No
|69|Yes

And you want to know if a person having age $19$ will be `eligible to vote` or not 

One way you can do is to make a decision, wether a `person age` greater than $18$ or not. 

We can visualize this as 

<img src = "https://media.istockphoto.com/id/1200922650/vector/decision-diagram-color-icon-block-chart-problem-solutions-operations-research-decision-tree.jpg?s=612x612&w=0&k=20&c=IjQiQfKAdYBpEDH3I_7L9uGKlLWIhWrhhwDglim4zFk=">

We call thee boxes `nodes`. Suppose the upper node is a `if-else` condition. If the condition is `True` we move to the `right` else to the `left`

So this was a small dataset. Lets assume we have a very big one. It would be way easier if we made a bigger version of the above image, something like this 

<img src = "https://static.vecteezy.com/system/resources/thumbnails/001/234/042/small/decision-tree-design.jpg">

At this point it is looking like a `tree` and we take `decisions` using this tree. So lets just name it `Decision Tree`. A decision tree basically is a gorup of nested `if-else` conditions. 

We call the lowermost `node` or where all the nodes `arose`, the `root node`, and the last nodes to be `leaf` nodes

The concept is rather to make `if-else` conditions manually, we somehow teach a machine to make `conditions` by looking at the data. 

But how can we tell if a `decision` is a good or bad?

Lets assume we have this dataset

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/sample-data/Sample_Data_1.csv
/kaggle/input/sample-data/Sample_Data_2.csv
/kaggle/input/sample-data/Sample_Data_3.csv


In [2]:
import numpy as np 
import pandas as pd 

In [3]:
data = pd.read_csv("/kaggle/input/sample-data/Sample_Data_1.csv")

In [4]:
data

,x,y,z
0,-33.963989,231.962014,a
1,34.152445,206.708388,a
2,7.545619,191.622019,a
3,6.862322,256.882087,a
4,-5.574613,209.725402,a
...,...,...,...
1158,630.098767,352.367983,b
1159,654.050088,412.010639,b
1160,709.733033,391.610226,b
1161,623.373461,405.861758,b


Lets now jsut focus on the `y` and the `z` column. 

In [5]:
data.drop("x" , axis = 1 , inplace = True)
data["z"] = np.where(data["z"] == "a" , 1 , 0)

Lets assume make a split at `300`. The condition will be 
```
data["y"] > 300
```

In [6]:
sample_1 = data[data["y"] > 300]
sample_2 = data[data["y"] < 300]

If we take a look at the values of `sample_1`

In [7]:
sample_1["z"].value_counts()

0    535
Name: z, dtype: int64

We have all unique values. At this point we can think as $300$ to be a good split point. but lets first also take a look at the other split

In [8]:
sample_2["z"].value_counts()

1    594
0     34
Name: z, dtype: int64

And we have a mixture of values in `sample_2` with the same split

We can see this is not a good split. But how can we tell computer that it is a bad split

For this we use `entorpy`. Entropy is basically the variance inside a node. First we calculate the ratio of every class in the nodes. If all the `leaf` nodes have only $1$ uniqe values. If the entropy is $0$, we call that a good split.

The formula for `Entropy`

$$E(S) = -\sum\limits_{i = 1}^{n}p_ilog(p_i)$$

Where $p = Ratio$

If we only see the `sample_1`. It is a perfect split. So its entropy will be 

In [9]:
-np.sum(np.array([p * np.log(p) 
                  for p in (np.bincount(sample_1["z"]) / sample_1.shape[0])]))

-0.0

If we do the same for `sample_2`

In [10]:
-np.sum(np.array([p * np.log(p) 
                  for p in (np.bincount(sample_2["z"]) / sample_2.shape[0])]))

0.21052969905042707

This shows that it is still a good split. but not the best split 

Now we know how to evaluate a split. But we still dont know when to stop spliting up the nodes. We still dont know if the node we got should be the last node or not. 

For this we use `Information Gain`, Basically we see if the entorpy of the child and parent was same or not. or basically we calculate the difference between them. 

$$Information_-Gain = E(Parent) - E(Child)$$

where $E = Entropy$

So now we know when to stop the spliting of the nodes. 

But we still dont know which value to choose for the split. Wether to use random or something else. 

Actually we try a split at every unique value of the data. To capture as much information as we can. 

Now we know about the Decision Trees. Lets try making a full algorithm for that 

First we need to make a `node` class

In [11]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        return self.value is not None

`Threshold` is a value where the split is taken 

Now we will make the `Decision Tree` class 

In [12]:
class DecisionTree:pass 

First we will define the `maximum depth` of the tree. Defining `Max_Depth` is important as decision trees can go infintly large

In [13]:
class DecisionTree:
    def __init__(self, max_depth = 100):
        self.max_depth = max_depth

Now we will add some more basic arguments to the tree

In [14]:
class DecisionTree:
    
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

A algorithm is useless without `fit` and `predict` method

In [15]:
class DecisionTree:
    
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None
        
    def fit(self , X , y):pass
    def predict(self , X):pass

Now we will make a helper funciton for calcuating `entropy` 

In [16]:
class DecisionTree:
    
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None
        
    def fit(self , X , y):pass
    
    entropy = lambda self , y : -np.sum(np.array([p * np.log(p)
                                                 for p in (np.bincount(y) / len(y))]))
    
    def predict(self , X):pass

Lets assume we have an array like this 

In [17]:
sample_array = np.arange(100)
sample_array 

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

And you want a $2$ seperate arrays containig the number greater then $50$ and less than $50$

In [18]:
np.argwhere(sample_array < 50).flatten()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

You could have written a proper code for this, but this is just a simple measure to do the same thing. 

A split will do the same thing we did before. Lets create a helper function for this too. We need a lot of help 

In [19]:
class DecisionTree:
    
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None
        
    def fit(self , X , y):pass
    
    entropy = lambda self , y : -np.sum(np.array([p * np.log(p)
                                                 for p in (np.bincount(y) / len(y))]))
    
    def split(self, X_column, split_thresh):
        
        left = np.argwhere(X_column <= split_thresh).flatten()
        right = np.argwhere(X_column > split_thresh).flatten()
        
        return left , right
    
    def predict(self , X):pass

Now we need to calulcate the `information_gain` at each split

In [20]:
class DecisionTree:
    
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None
        
    def fit(self , X , y):pass
    
    def _information_gain(self, y, X_column, threshold):

        parent_entropy = self._entropy(y)

        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        child_entropy = (len(left_idxs)/len(y)) * (self.entropy(y[left_idxs])) + (len(right_idxs)/len(y)) * (self._entropy(y[right_idxs]))

        information_gain = parent_entropy - child_entropy
        return information_gain
    
    entropy = lambda self , y : -np.sum(np.array([p * np.log(p)
                                                 for p in (np.bincount(y) / len(y))]))
    
    def split(self, X_column, split_thresh):
        
        left = np.argwhere(X_column <= split_thresh).flatten()
        right = np.argwhere(X_column > split_thresh).flatten()
        
        return left , right
    
    def predict(self , X):pass

Now we need to evaluate the `information_gain`  and get the `best split`

In [21]:
class DecisionTree:
    
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None
        
    def fit(self , X , y):pass
    
    def _best_split(self, X, y, feat_idxs):
        
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold
    
    def _information_gain(self, y, X_column, threshold):

        parent_entropy = self._entropy(y)

        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        child_entropy = (len(left_idxs)/len(y)) * (self.entropy(y[left_idxs])) + (len(right_idxs)/len(y)) * (self._entropy(y[right_idxs]))

        information_gain = parent_entropy - child_entropy
        return information_gain
    
    entropy = lambda self , y : -np.sum(np.array([p * np.log(p)
                                                 for p in (np.bincount(y) / len(y))]))
    
    def split(self, X_column, split_thresh):
        
        left = np.argwhere(X_column <= split_thresh).flatten()
        right = np.argwhere(X_column > split_thresh).flatten()
        
        return left , right
    
    def predict(self , X):pass

Now we just need to do this iteratively till some `stopping condition`

In [22]:
class DecisionTree:
    
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None
        
    def fit(self , X , y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)
    
    def _grow_tree(self, X, y, depth=0):
        
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)
    
    def _best_split(self, X, y, feat_idxs):
        
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold
    
    def _information_gain(self, y, X_column, threshold):

        parent_entropy = self._entropy(y)

        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        child_entropy = (len(left_idxs)/len(y)) * (self.entropy(y[left_idxs])) + (len(right_idxs)/len(y)) * (self._entropy(y[right_idxs]))

        information_gain = parent_entropy - child_entropy
        return information_gain
    
    entropy = lambda self , y : -np.sum(np.array([p * np.log(p)
                                                 for p in (np.bincount(y) / len(y))]))
    
    def split(self, X_column, split_thresh):
        
        left = np.argwhere(X_column <= split_thresh).flatten()
        right = np.argwhere(X_column > split_thresh).flatten()
        
        return left , right
    
    def predict(self , X):pass

We have made most of our algorithm for `Decision Tree`. Now we just need to add a function so that it can predict

In [23]:
class DecisionTree:
    
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None
        
    def fit(self , X , y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)
    
    def _grow_tree(self, X, y, depth=0):
        
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)
    
    def _best_split(self, X, y, feat_idxs):
        
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold
    
    def _information_gain(self, y, X_column, threshold):

        parent_entropy = self._entropy(y)

        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        child_entropy = (len(left_idxs)/len(y)) * (self.entropy(y[left_idxs])) + (len(right_idxs)/len(y)) * (self._entropy(y[right_idxs]))

        information_gain = parent_entropy - child_entropy
        return information_gain
    
    entropy = lambda self , y : -np.sum(np.array([p * np.log(p)
                                                 for p in (np.bincount(y) / len(y))]))
    
    def split(self, X_column, split_thresh):
        
        left = np.argwhere(X_column <= split_thresh).flatten()
        right = np.argwhere(X_column > split_thresh).flatten()
        
        return left , right
    
    def _most_common_label(self, y):
        
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        
        return value

    def predict(self, X):
        
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        
        if node.is_leaf_node():
            
            return node.value

        if x[node.feature] <= node.threshold:
            
            return self._traverse_tree(x, node.left)
        
        return self._traverse_tree(x, node.right)

And we have finally made our `Decision Tree`

<img src = "https://media.makeameme.org/created/finally-a-decision.jpg">

# Decision Trees Final Source Code

In [24]:
class Node:
    
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        
        return self.value is not None

class DecisionTree:
    
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None
        
    def fit(self , X , y):
        
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)
    
    def _grow_tree(self, X, y, depth=0):
        
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            
            leaf_value = self._most_common_label(y)
            
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)
    
    def _best_split(self, X, y, feat_idxs):
        
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold
    
    def _information_gain(self, y, X_column, threshold):

        parent_entropy = self._entropy(y)

        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            
            return 0

        child_entropy = (len(left_idxs)/len(y)) * (self.entropy(y[left_idxs])) + (len(right_idxs)/len(y)) * (self._entropy(y[right_idxs]))

        information_gain = parent_entropy - child_entropy
        return information_gain
    
    entropy = lambda self , y : -np.sum(np.array([p * np.log(p)
                                                 for p in (np.bincount(y) / len(y))]))
    
    def split(self, X_column, split_thresh):
        
        left = np.argwhere(X_column <= split_thresh).flatten()
        right = np.argwhere(X_column > split_thresh).flatten()
        
        return left , right
    
    def _most_common_label(self, y):
        
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        
        return value

    def predict(self, X):
        
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        
        if node.is_leaf_node():
            
            return node.value

        if x[node.feature] <= node.threshold:
            
            return self._traverse_tree(x, node.left)
        
        return self._traverse_tree(x, node.right)

**THIS IS NOT THE FULL IMPLEMENTATION, IT STILL LACKS MANY FUNCTIONALITIES AND IS VULENRABLE TO MANY EDGE CASES, WE WILL IMPROVE THIS IN THE UPCOMING VERSIONS**

**PLEASE COMMENT DOWN IF I DID ANY MISTAKES, OR IF CAN MAKE THIS MORE CONNECTED TO THE GROUND, OR SUGGESTIONS. YOUR ASSISTS ARE HIGHLY APPRECIABLE**

**THATS IT FOR TODAY GUYS**

**HOPE YOU UNDERSTOOD AND LIKED MY WORK**

**DONT FORGET TO MAKE AN UPVOTE :)**

**PEACE OUT !!!**